# Random pin dropping in GEE

This script will aim to drop random pins on a map within a bounding box. The user will then be able to "classify" the dropped pins based on the image.

In [1]:
# initial import of GEE
import ee

ee.Initialize()

In [17]:
import numpy as np
import folium # for interactive maps!
import geehydro # unsure what geehydro does...
from IPython.display import Image # for displaying the Images
import IPython.display as IPyDisplay
import random as rn
import pandas as pd

In [3]:
# initialise the parameters within which we want to select random points
pins_bbox = [58.4,-4.1,58.6,-3.8]
pins_mapscale = 12


classifications = {'forest':'q', 'peat':'e','farmland':'t'}

pins = []

class Pin:
    def __init__(self,coords,class_):
        self.coords = coords
        self.classification = class_
    def __str__(self):
        return '( {:.3}, {:.3} ) : {}'.format(*self.coords,self.classification)

map_centre = [ (pins_bbox[0]+pins_bbox[2])/2, (pins_bbox[1]+pins_bbox[3])/2 ]

first = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(ee.Geometry.Point(*map_centre[::-1])).filterDate('2018-01-01', '2018-12-31').sort('CLOUDY_PIXEL_PERCENTAGE').first() # folium and ee use different order in lat-lon coordinates.
Image(url=first.getThumbUrl({'min': 0, 'max': 2000, 'bands':['B4','B3','B2'],'dimensions': 512}))

In [13]:
# map initialisation. Can be reran to reset the map.
map = folium.Map(location=map_centre,zoom_start=pins_mapscale)

map.addLayer(first, {'min': 0, 'max': 2000, 'bands':['B4','B3','B2']}, 'Colour Image')

folium.Rectangle(bounds=[tuple(pins_bbox[:2]),tuple(pins_bbox[2:])],
                colour='red',
                weight=5,
                fill=False
                ).add_to(map)

display(map)

# Dynamic maps with folium
Aren't possible. In order to get around this, I will create a single cell that, once setup is complete, should be repeatedly ran again and again. The cell above can be ran in order to visualise all the pins that have been generated. Once enough pins have been generated and classified, we will then run a cell (currently hypothetical) to save the pins that can then be loaded into the classifying program.

In [14]:
# pin creation cell
map.location = map_centre
pins_n = 10 # number of pins to be classified

#marker = folium.Marker(location=map_centre).add_to(map)

def randomCoordInBBox(bbox):
    # function that will return a random (lat,lon) coordinate within a bounding box
    # INPUT bbox: [4x1] list, (lat1,lon1,lat2,lon2)
    # RETURNS coord: (lat, lon)
    blat = [bbox[0], bbox[2]]
    blon = [bbox[1], bbox[3]]
    rlat = rn.random()*(blat[1]-blat[0]) + blat[0]
    rlon = rn.random()*(blon[1]-blon[0]) + blon[0]
    return [rlat,rlon]

def classify():
    allowedInputs = {'p':'plantation',
                    'b':'burnt',
                    'l':'peatland',
                    'a':'agriculture',
                    'c':'cleared',
                    's':'sea',
                    't':'other'
                    }
    validClass = False
    while ~validClass:
        inp = input('Class for pin: ')
        if inp in allowedInputs:
            validClass = True
            return allowedInputs[inp]

for i in range(pins_n):
    centre = randomCoordInBBox(pins_bbox)
    print(centre)
    map.location = centre
    
    folium.Marker(location=centre).add_to(map)

    IPyDisplay.clear_output()
    display(map)
    class_ = classify()
    pins.append(Pin(centre,class_))

# after everything, save pins


In [19]:
# turn the pins into a pandas object, and save as a csv.
allowedInputs = ['plantation',
                'burnt',
                'peatland',
                'agriculture',
                'cleared',
                'sea',
                'other']

lat = []
lon = []
classification = []

for p in pins:
    lat.append(p.coords[0])
    lon.append(p.coords[1])
    classification.append(allowedInputs.index(p.classification))

d = {'lat':lat,'lon':lon,'classification':classification}
df = pd.DataFrame(data=d)
df
#df.to_csv('initTest.csv')